#### Licenses
GloVe
Public Domain Dedication and License v1.0 whose full text can be found at: http://www.opendatacommons.org/licenses/pddl/1.0/

Facebookresearch / FastText words embeddings
https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md

@article{bojanowski2016enriching,
  title={Enriching Word Vectors with Subword Information},
  author={Bojanowski, Piotr and Grave, Edouard and Joulin, Armand and Mikolov, Tomas},
  journal={arXiv preprint arXiv:1607.04606},
  year={2016}
}

License Creative Commons Attribution-Share-Alike License 3.0 (https://creativecommons.org/licenses/by-sa/3.0/)

In [1]:
# optional - plays a sound when a cell completed
# note: for any reason this should be executed after keras imports

from time import time
from IPython import get_ipython
from IPython.display import Audio, display


class InvisibleAudio(Audio):
    def _repr_html_(self):
        audio = super()._repr_html_()
        audio = audio.replace('<audio', f'<audio onended="this.parentNode.removeChild(this)"')
        return f'<div style="display:none">{audio}</div>'

class Beeper:

    def __init__(self, threshold, **audio_kwargs):
        self.threshold = threshold
        self.start_time = None    # time in sec, or None
        self.audio = audio_kwargs

    def pre_execute(self):
        if not self.start_time:
            self.start_time = time()

    def post_execute(self):
        end_time = time()
        if self.start_time and end_time - self.start_time > self.threshold:
            audio = InvisibleAudio(**self.audio, autoplay=True)
            display(audio)
        self.start_time = None


beeper = Beeper(5, url='http://www.soundjay.com/button/beep-07.wav')

ipython = get_ipython()
ipython.events.register('pre_execute', beeper.pre_execute)
ipython.events.register('post_execute', beeper.post_execute)

In [2]:
import os
import io
import pickle
import numpy as np
from keras.utils import to_categorical

class DataManager:
    
    root_dir_ = '.'
    
    
    def __init__(self):
        self.root_dir_ = os.getcwd()
        
    def load_dummy_data(self):
        """
        This method makes available some dummy training data.
        """
        X = self.pickle_load("data/fr_X.pkl")
        Y = self.pickle_load("data/fr_Y.pkl")
        vocab_mots = self.pickle_load("data/fr_vocab_mots.pkl")
        vocab_pdd = self.pickle_load("data/fr_vocab_pdd.pkl")
        vocab_liaisons = self.pickle_load("data/fr_vocab_liaisons.pkl")
        return X, Y, vocab_mots, vocab_pdd, vocab_liaisons
        
    def load_dummy_data_2(self):
        """
        This method makes available some dummy training data.
        """
        data = self.pickle_load("data/f1_fr_project_ok_bool.pkl")
        return data['X'], data['Y'], data['vocab_mots'], data['vocab_pdd'], data['vocab_liaisons'] 
    
    def load_data(self, phase='train', lang='fr', featureset='f1'):
        """
        Loads a dataset for a specific lang and feature set, and phase (train/dev/test).
        
        Parameters
        ----------
        
        phase: str
            'train', 'dev' or 'test'
            
        lang: str
        
        featureset: str
            'f1', 'f2' or 'f3'
        
        """
        name = "{featureset}_{lang}-{phase}".format(lang=lang, featureset=featureset, phase=phase)
        fname = os.path.join('data', name)
        data = self.pickle_load(fname)
        if data:
            vocabs = {}
            X = np.array(data['X'])
            Y = np.array(data['Y'])
            vocabs['WORDS'] = data['vocab_mots']
            vocabs['POS'] = data['vocab_pdd']
            vocabs['LABELS'] = data['vocab_liaisons']
            if isinstance(vocabs['WORDS'], np.ndarray):
                vocabs['WORDS'] = vocabs['WORDS'].tolist()
            if isinstance(vocabs['POS'], np.ndarray):
                vocabs['POS'] = vocabs['POS'].tolist()
            if isinstance(vocabs['LABELS'], np.ndarray):
                vocabs['LABELS'] = vocabs['LABELS'].tolist()
                
            if featureset == 'f2' or featureset == 'f3':
                vocabs['MORPHO'] = data['vocab_morpho']
                vocabs['LEMMA'] = data['vocab_lemma']
                if isinstance(vocabs['MORPHO'], np.ndarray):
                    vocabs['MORPHO'] = vocabs['MORPHO'].tolist()
                if isinstance(vocabs['LEMMA'], np.ndarray):
                    vocabs['LEMMA'] = vocabs['LEMMA'].tolist()
                       
            print("load_data: loaded X = ", X.shape, ", Y = ", Y.shape, ", vocabs = ", 
                  (''.join("{key} ({len}), ".format(key=k, len=len(vocabs[k])) for k in vocabs.keys())))
            return X, Y, vocabs
        
        return None
    
    def merge_vocabs(self, vocab1, vocab2, data, columns, verbose=False):
        """
        Merges vocab2 into vocab1, updating accordingly words references in data columns.
        For all words in vocab2 with index idx_vocab2:
        - if it exists in vocab1, then words features of data in columns refering to idx_vocab2 will be replaced by idx_vocab1
        - if it does not exist in vocab1, it will be appended to vocab1 then replacement will be done as above in data
        
        Parameters
        ----------
        
        vocab1: list(str)
            A list of words from original vocabulary.
                        
        vocab2: list(str)
            A list of words from vocabulary to be merged into vocab1 (used to build X2).
            
        data: array
            Array containing the words indices to be updated by merge of vocabularies.
            Usually this should be an array with samples as first dimension (rows), then columns for features.

        columns: tuple(int)
            Indices of columns to be updated in data.
            
        Returns:
        
        vocab1: list(str)
            A new vocabulary with missing words appended to vocab1.
            Note: vocab1 is also update in place, meaning this function modified original vocab1.
        
        ! data is updated in place.
        
        """
        
        vocab1_ = vocab1.copy()
        data_ = data.copy()
        
        for idx_vocab2, w in enumerate(vocab2):
            # treat vocabs
            if w in vocab1:
                if verbose:
                    print("word [{i}]={w} found in vocab1 at {j}".format(i=idx_vocab2, w=w, j=vocab1.index(w)))
                idx_vocab1 = vocab1.index(w)
            else:
                if verbose:
                    print("word [{i}]={w} not found in vocab1".format(i=idx_vocab2, w=w))
                vocab1.append(w)
                idx_vocab1 = len(vocab1) - 1
                
            # replace all references in data
            for i in range(len(data_)):
                if len(data_.shape) == 1:
                    if data_[i] == idx_vocab2:
                        if verbose:
                            print("Replacing word [{i}]= {idx}, {w} index {fro} to {to}"
                                    .format(i=i, idx=idx_vocab2, w=vocab2[idx_vocab2], 
                                            fro=idx_vocab2, to=idx_vocab1))
                        data[i] = idx_vocab1
                        #Replacing word [796]= 1, LEFT_advcl index 1 to 2
                        
                else:
                    for idx, j in enumerate(columns):
                        if data_[i][j] == idx_vocab2:
                            if verbose:
                                print("Replacing word [{i},{j}]= {idx}, {w} index {fro} to {to}"
                                      .format(i=i, j=j, idx=idx_vocab2, w=vocab2[idx_vocab2], 
                                              fro=idx_vocab2, to=idx_vocab1))
                            data[i][j] = idx_vocab1

        return vocab1, data
        

    # some utilities for saving results
    def safe_pickle_dump(self, filename, obj):
        """
        Serializes an object to file, creating directory structure if it does not exist.
        """
        name = filename
        print("Saving "+name)
        try:
            os.makedirs(os.path.dirname(name), exist_ok=True)
            f = open(name,"wb")
            pickle.dump(obj,f)
            f.close()
        except:
            return False
    
        return True
    
    def pickle_load(self, filename):
        """
        Deserialize an object from a file created with pickle.dump.
        Returns False if this failed.
        """
        name = filename
        print("Loading "+name)
        try:
            f = open(name,"rb")
            obj = pickle.load(f)
            f.close()
            return obj
        except Exception as e:
            print(e)
            return False
      
        return True
    

    
    def load_glove_embeddings(self):
        EMBEDDING_DIM = 100
        GLOVE_DIR='Gloves/dataset'
        f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'), encoding="utf8")

        embeddings_index = {}
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
    
            embeddings_index[word] = coefs
        f.close()
        print('Found %s word vectors.' % len(embeddings_index))
        
    def load_embeddings(self, lang, type='fasttext'):
        """
        Loads an embeddings file depending on its type and language.
        
        Parameters
        ----------
        
        type: str
            Only "fasttext" is supported.
            
        lang: str
            See load_fasttext_embeddings(lang)
        
        """
        
        return self.load_fasttext_embeddings(lang)
        
    def load_fasttext_embeddings(self, lang):
        """
        Loads a fasttext embedding, chosen depending on lang parameter provided.
        File expected as root_dir_/data/embeddings/facebookresearch/wiki.{lang}.vec
        (or as root_dir_/data/embeddings/facebookresearch/wiki.{lang}.vec.pkl if already loaded once)
        
        Parameters
        ----------
        
        lang: str
            One of 'fr', 'ja', 'en', 'nl' (or additional ones depending on embeddings present on disk).
        
        """
        data_dict = {}
        
        pickle_fname = "wiki.{lang}.vec.pkl".format(lang=lang)
        pickle_ffname = os.path.join(self.root_dir_, 'data', 'embeddings', 'facebookresearch', pickle_fname)
        
        if os.path.isfile(pickle_ffname):
            data_dict = self.pickle_load(pickle_ffname)
            print("Embedding for {lang} loaded from {fname}".format(lang=lang, fname=pickle_ffname))
        else: 
            fname = "wiki.{lang}.vec".format(lang=lang)
            data_file = os.path.join(self.root_dir_, 'data', 'embeddings', 'facebookresearch', fname)
        
            fin = io.open(data_file, 'r', encoding='utf-8', newline='\n', errors='ignore')
            n, d = map(int, fin.readline().split())
            
            for line in fin:
                tokens = line.rstrip().split(' ')
                data_dict[tokens[0]] = list(map(float, tokens[1:]))
            print("Embedding for {lang} loaded from {fname}".format(lang=lang, fname=data_file))
            # save embeddings as array format to improve speed next time
            self.safe_pickle_dump(pickle_ffname, data_dict)

        print("Read ", len(data_dict), " words vectors")
        return data_dict
    
    def get_words_to_match_for_embeddings(self, word):
        return [word, word.lower(), word.lower().replace('-', ''), word.lower().replace('-', '').replace('\\xa0', ' ')]
    
    def align_embeddings(self, vocab, embeddings):
        """
        Generates aligned embeddings from original embeddings, and a vocabulary of words.
        Words from vocabulary may not exist in original embeddings, in this case a random vector is generated.
        Words matching is done as (by priority) : exact match, then case insensitively, then with dash ('-') removed.
        
        Parameters
        ----------
        
        vocab: array
            An array containing each word in the vocabulary.
            
        embeddings: dict
            A dict object with words as keys and their embeddings (as a vector array) as values.
            
        Returns
        -------
        
        aligned_embeddings: list
            An array containing:
          - for words from vocab found in embeddings, the corresponding embedding at same index as in vocab.
          - for words not found in embeddings, a random vector, at same index as in vocab.
        
        words_not_found: list
            An array containing indices (based on vocab) of words not found in embeddings and replaced by random
            values.
            
        words_matched: list
            An array of strings of words based on vocab, as they were matched in embeddings.
            For example if lowercased word from vocab was matched, then lowercase version of this word will be found
            in this table (whereas the original case sensitive word will remain as is in vocab array)
        
        """
        dim_embedding = len(embeddings[list(embeddings.keys())[0]]) # find length from value of 'first' key
        aligned_embeddings = np.zeros((len(vocab), dim_embedding))
        words_not_found = []
        words_matched = [None] * len(vocab)
        for idx_mot, mot in enumerate(vocab):
            words_to_match = self.get_words_to_match_for_embeddings(mot)
            for word_to_match in words_to_match:
                if word_to_match in embeddings.keys():
                    aligned_embeddings[idx_mot] = embeddings[word_to_match]
                    words_matched[idx_mot] = word_to_match
                    break
            if words_matched[idx_mot] is None:
                words_not_found.append(idx_mot)
                words_matched[idx_mot] = mot
                aligned_embeddings[idx_mot] = np.random.rand(dim_embedding)
                
        return aligned_embeddings, words_not_found, words_matched
    
dm = DataManager()

C:\Users\jerem\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [147]:
X_train, Y_train, vocabs_train = dm.load_data()
X_dev, Y_dev, vocabs_dev = dm.load_data(phase='dev')
X_test, Y_test, vocabs_test = dm.load_data(phase='test')

Loading data\f1_fr-train
load_data: loaded X =  (679243, 6) , Y =  (679243,) , vocabs =  WORDS (42278), POS (20), LABELS (90), 
Loading data\f1_fr-dev
load_data: loaded X =  (68105, 6) , Y =  (68105,) , vocabs =  WORDS (9275), POS (20), LABELS (74), 
Loading data\f1_fr-test
load_data: loaded X =  (19119, 6) , Y =  (19119,) , vocabs =  WORDS (3285), POS (20), LABELS (68), 


In [61]:
deps_in_dev_only = [d for d in vocabs_dev['DEPS'] if d not in vocabs_train['DEPS']]
print("Dependencies in dev not in train ", len(deps_in_dev_only))
deps_in_train_only = [d for d in vocabs_train['DEPS'] if d not in vocabs_dev['DEPS']]
print("Dependencies in train not in dev ", len(deps_in_train_only))
deps_in_test_only = [d for d in vocabs_test['DEPS'] if d not in vocabs_train['DEPS']]
print("Dependencies in test not in train ", len(deps_in_test_only))

print(vocabs_train['DEPS'][67])
print(vocabs_dev['DEPS'][67])
print(vocabs_test['DEPS'][67])

Dependencies in dev not in train  0
Dependencies in train not in dev  16
Dependencies in test not in train  0
RIGHT_flat:name
RIGHT_obl:mod
SHIFT


In [41]:
print(len(vocabs_train['WORDS']))
in_dev_only = [w for w in vocabs_dev['WORDS'] if w not in vocabs_train['WORDS']]
print ("words in dev not in train ", len(in_dev_only))
in_test_only = [w for w in vocabs_test['WORDS'] if w not in vocabs_train['WORDS']]
print ("words in test not in train ", len(in_test_only))
in_test_only_ = [w for w in vocabs_test['WORDS'] if w not in vocabs_train['WORDS'] and w not in vocabs_dev['WORDS']]
print ("words in test not in train nor dev ", len(in_test_only_))

vocabs_train_ = {}
vocabs_dev_ = {}
vocabs_train_['WORDS'] = [w.lower().replace('-', '').replace('\\xa0', ' ') for w in vocabs_train['WORDS']]
vocabs_dev_['WORDS'] = [w.lower().replace('-', '').replace('\\xa0', ' ') for w in vocabs_dev['WORDS']]
in_dev_only_r = [w for w in vocabs_dev_['WORDS'] if w not in vocabs_train_['WORDS']]
print ("words in dev not in train (after words cleaning)", len(in_dev_only_r))



44863
words in dev not in train  0
words in test not in train  560
words in test not in train nor dev  560
words in dev not in train (after words cleaning) 0
(68105, 6)


In [140]:
dm.merge_vocabs(vocab1=vocabs_train['WORDS'], vocab2=vocabs_dev['WORDS'], data=X_dev, columns=(0, 1))
print(len(vocabs_train['WORDS']))

44863


In [138]:
print(Y_test)

print("  ... merge and align LABELS (Y) of dev into train ...")
dm.merge_vocabs(vocab1=vocabs_train['DEPS'], vocab2=vocabs_dev['DEPS'], data=Y_dev, columns=(0,))
print("  ... merge and align LABELS (Y) of test into train ...")
dm.merge_vocabs(vocab1=vocabs_train['DEPS'], vocab2=vocabs_test['DEPS'], data=Y_test, columns=(0,), verbose=True)

print(Y_test)


[67 23 65 ... 40 34 64]
  ... merge and align LABELS (Y) of dev into train ...
  ... merge and align LABELS (Y) of test into train ...
word [0]=LEFT_acl found in vocab1 at 0
Replacing word [2154]= 0, LEFT_acl index 0 to 0
Replacing word [2798]= 0, LEFT_acl index 0 to 0
Replacing word [5409]= 0, LEFT_acl index 0 to 0
Replacing word [7714]= 0, LEFT_acl index 0 to 0
Replacing word [16306]= 0, LEFT_acl index 0 to 0
word [1]=LEFT_advcl found in vocab1 at 2
Replacing word [796]= 1, LEFT_advcl index 1 to 2
Replacing word [911]= 1, LEFT_advcl index 1 to 2
Replacing word [1370]= 1, LEFT_advcl index 1 to 2
Replacing word [2062]= 1, LEFT_advcl index 1 to 2
Replacing word [2624]= 1, LEFT_advcl index 1 to 2
Replacing word [2744]= 1, LEFT_advcl index 1 to 2
Replacing word [3404]= 1, LEFT_advcl index 1 to 2
Replacing word [3480]= 1, LEFT_advcl index 1 to 2
Replacing word [3682]= 1, LEFT_advcl index 1 to 2
Replacing word [3848]= 1, LEFT_advcl index 1 to 2
Replacing word [4108]= 1, LEFT_advcl index 1 t

Replacing word [14492]= 8, LEFT_case index 8 to 9
Replacing word [14495]= 8, LEFT_case index 8 to 9
Replacing word [14499]= 8, LEFT_case index 8 to 9
Replacing word [14513]= 8, LEFT_case index 8 to 9
Replacing word [14516]= 8, LEFT_case index 8 to 9
Replacing word [14521]= 8, LEFT_case index 8 to 9
Replacing word [14552]= 8, LEFT_case index 8 to 9
Replacing word [14562]= 8, LEFT_case index 8 to 9
Replacing word [14581]= 8, LEFT_case index 8 to 9
Replacing word [14588]= 8, LEFT_case index 8 to 9
Replacing word [14598]= 8, LEFT_case index 8 to 9
Replacing word [14601]= 8, LEFT_case index 8 to 9
Replacing word [14609]= 8, LEFT_case index 8 to 9
Replacing word [14615]= 8, LEFT_case index 8 to 9
Replacing word [14650]= 8, LEFT_case index 8 to 9
Replacing word [14655]= 8, LEFT_case index 8 to 9
Replacing word [14672]= 8, LEFT_case index 8 to 9
Replacing word [14688]= 8, LEFT_case index 8 to 9
Replacing word [14694]= 8, LEFT_case index 8 to 9
Replacing word [14710]= 8, LEFT_case index 8 to 9


Replacing word [9091]= 15, LEFT_det index 15 to 16
Replacing word [9096]= 15, LEFT_det index 15 to 16
Replacing word [9109]= 15, LEFT_det index 15 to 16
Replacing word [9130]= 15, LEFT_det index 15 to 16
Replacing word [9135]= 15, LEFT_det index 15 to 16
Replacing word [9155]= 15, LEFT_det index 15 to 16
Replacing word [9177]= 15, LEFT_det index 15 to 16
Replacing word [9190]= 15, LEFT_det index 15 to 16
Replacing word [9211]= 15, LEFT_det index 15 to 16
Replacing word [9224]= 15, LEFT_det index 15 to 16
Replacing word [9243]= 15, LEFT_det index 15 to 16
Replacing word [9250]= 15, LEFT_det index 15 to 16
Replacing word [9263]= 15, LEFT_det index 15 to 16
Replacing word [9286]= 15, LEFT_det index 15 to 16
Replacing word [9310]= 15, LEFT_det index 15 to 16
Replacing word [9325]= 15, LEFT_det index 15 to 16
Replacing word [9332]= 15, LEFT_det index 15 to 16
Replacing word [9352]= 15, LEFT_det index 15 to 16
Replacing word [9357]= 15, LEFT_det index 15 to 16
Replacing word [9379]= 15, LEFT

Replacing word [13181]= 23, LEFT_nsubj index 23 to 26
Replacing word [13229]= 23, LEFT_nsubj index 23 to 26
Replacing word [13242]= 23, LEFT_nsubj index 23 to 26
Replacing word [13312]= 23, LEFT_nsubj index 23 to 26
Replacing word [13326]= 23, LEFT_nsubj index 23 to 26
Replacing word [13345]= 23, LEFT_nsubj index 23 to 26
Replacing word [13366]= 23, LEFT_nsubj index 23 to 26
Replacing word [13390]= 23, LEFT_nsubj index 23 to 26
Replacing word [13458]= 23, LEFT_nsubj index 23 to 26
Replacing word [13476]= 23, LEFT_nsubj index 23 to 26
Replacing word [13552]= 23, LEFT_nsubj index 23 to 26
Replacing word [13571]= 23, LEFT_nsubj index 23 to 26
Replacing word [13606]= 23, LEFT_nsubj index 23 to 26
Replacing word [13688]= 23, LEFT_nsubj index 23 to 26
Replacing word [13728]= 23, LEFT_nsubj index 23 to 26
Replacing word [13772]= 23, LEFT_nsubj index 23 to 26
Replacing word [13814]= 23, LEFT_nsubj index 23 to 26
Replacing word [13842]= 23, LEFT_nsubj index 23 to 26
Replacing word [13914]= 23, 

Replacing word [1073]= 34, REDUCE index 34 to 41
Replacing word [1079]= 34, REDUCE index 34 to 41
Replacing word [1082]= 34, REDUCE index 34 to 41
Replacing word [1083]= 34, REDUCE index 34 to 41
Replacing word [1095]= 34, REDUCE index 34 to 41
Replacing word [1102]= 34, REDUCE index 34 to 41
Replacing word [1112]= 34, REDUCE index 34 to 41
Replacing word [1113]= 34, REDUCE index 34 to 41
Replacing word [1114]= 34, REDUCE index 34 to 41
Replacing word [1115]= 34, REDUCE index 34 to 41
Replacing word [1128]= 34, REDUCE index 34 to 41
Replacing word [1129]= 34, REDUCE index 34 to 41
Replacing word [1131]= 34, REDUCE index 34 to 41
Replacing word [1140]= 34, REDUCE index 34 to 41
Replacing word [1153]= 34, REDUCE index 34 to 41
Replacing word [1154]= 34, REDUCE index 34 to 41
Replacing word [1155]= 34, REDUCE index 34 to 41
Replacing word [1170]= 34, REDUCE index 34 to 41
Replacing word [1171]= 34, REDUCE index 34 to 41
Replacing word [1187]= 34, REDUCE index 34 to 41
Replacing word [1193

...

Replacing word [2009]= 51, RIGHT_flat:name index 51 to 67
Replacing word [2482]= 51, RIGHT_flat:name index 51 to 67
Replacing word [2484]= 51, RIGHT_flat:name index 51 to 67
Replacing word [2486]= 51, RIGHT_flat:name index 51 to 67
Replacing word [2491]= 51, RIGHT_flat:name index 51 to 67
Replacing word [2493]= 51, RIGHT_flat:name index 51 to 67
Replacing word [2514]= 51, RIGHT_flat:name index 51 to 67
Replacing word [2643]= 51, RIGHT_flat:name index 51 to 67
Replacing word [2648]= 51, RIGHT_flat:name index 51 to 67
Replacing word [3354]= 51, RIGHT_flat:name index 51 to 67
Replacing word [3356]= 51, RIGHT_flat:name index 51 to 67
Replacing word [3358]= 51, RIGHT_flat:name index 51 to 67
Replacing word [4139]= 51, RIGHT_flat:name index 51 to 67
Replacing word [4156]= 51, RIGHT_flat:name index 51 to 67
Replacing word [4291]= 51, RIGHT_flat:name index 51 to 67
Replacing word [4339]= 51, RIGHT_flat:name index 51 to 67
Replacing word [4507]= 51, RIGHT_flat:name index 51 to 67
Replacing word

Replacing word [17412]= 59, RIGHT_obl index 59 to 79
Replacing word [17512]= 59, RIGHT_obl index 59 to 79
Replacing word [17561]= 59, RIGHT_obl index 59 to 79
Replacing word [17669]= 59, RIGHT_obl index 59 to 79
Replacing word [17921]= 59, RIGHT_obl index 59 to 79
Replacing word [17925]= 59, RIGHT_obl index 59 to 79
Replacing word [17956]= 59, RIGHT_obl index 59 to 79
Replacing word [18002]= 59, RIGHT_obl index 59 to 79
Replacing word [18069]= 59, RIGHT_obl index 59 to 79
Replacing word [18113]= 59, RIGHT_obl index 59 to 79
Replacing word [18121]= 59, RIGHT_obl index 59 to 79
Replacing word [18231]= 59, RIGHT_obl index 59 to 79
Replacing word [18249]= 59, RIGHT_obl index 59 to 79
Replacing word [18296]= 59, RIGHT_obl index 59 to 79
Replacing word [18343]= 59, RIGHT_obl index 59 to 79
Replacing word [18357]= 59, RIGHT_obl index 59 to 79
Replacing word [18395]= 59, RIGHT_obl index 59 to 79
Replacing word [18594]= 59, RIGHT_obl index 59 to 79
Replacing word [18617]= 59, RIGHT_obl index 59

Replacing word [13620]= 66, RIGHT_xcomp index 66 to 88
Replacing word [14135]= 66, RIGHT_xcomp index 66 to 88
Replacing word [14153]= 66, RIGHT_xcomp index 66 to 88
Replacing word [14171]= 66, RIGHT_xcomp index 66 to 88
Replacing word [14579]= 66, RIGHT_xcomp index 66 to 88
Replacing word [15333]= 66, RIGHT_xcomp index 66 to 88
Replacing word [15402]= 66, RIGHT_xcomp index 66 to 88
Replacing word [15543]= 66, RIGHT_xcomp index 66 to 88
Replacing word [15578]= 66, RIGHT_xcomp index 66 to 88
Replacing word [15935]= 66, RIGHT_xcomp index 66 to 88
Replacing word [15938]= 66, RIGHT_xcomp index 66 to 88
Replacing word [16001]= 66, RIGHT_xcomp index 66 to 88
Replacing word [16042]= 66, RIGHT_xcomp index 66 to 88
Replacing word [16266]= 66, RIGHT_xcomp index 66 to 88
Replacing word [16279]= 66, RIGHT_xcomp index 66 to 88
Replacing word [16351]= 66, RIGHT_xcomp index 66 to 88
Replacing word [16982]= 66, RIGHT_xcomp index 66 to 88
Replacing word [17189]= 66, RIGHT_xcomp index 66 to 88
Replacing 

Replacing word [6724]= 67, SHIFT index 67 to 89
Replacing word [6734]= 67, SHIFT index 67 to 89
Replacing word [6735]= 67, SHIFT index 67 to 89
Replacing word [6741]= 67, SHIFT index 67 to 89
Replacing word [6742]= 67, SHIFT index 67 to 89
Replacing word [6743]= 67, SHIFT index 67 to 89
Replacing word [6745]= 67, SHIFT index 67 to 89
Replacing word [6746]= 67, SHIFT index 67 to 89
Replacing word [6752]= 67, SHIFT index 67 to 89
Replacing word [6753]= 67, SHIFT index 67 to 89
Replacing word [6760]= 67, SHIFT index 67 to 89
Replacing word [6762]= 67, SHIFT index 67 to 89
Replacing word [6766]= 67, SHIFT index 67 to 89
Replacing word [6769]= 67, SHIFT index 67 to 89
Replacing word [6775]= 67, SHIFT index 67 to 89
Replacing word [6776]= 67, SHIFT index 67 to 89
Replacing word [6777]= 67, SHIFT index 67 to 89
Replacing word [6783]= 67, SHIFT index 67 to 89
Replacing word [6789]= 67, SHIFT index 67 to 89
Replacing word [6790]= 67, SHIFT index 67 to 89
Replacing word [6796]= 67, SHIFT index 6

Replacing word [13608]= 67, SHIFT index 67 to 89
Replacing word [13609]= 67, SHIFT index 67 to 89
Replacing word [13613]= 67, SHIFT index 67 to 89
Replacing word [13614]= 67, SHIFT index 67 to 89
Replacing word [13615]= 67, SHIFT index 67 to 89
Replacing word [13623]= 67, SHIFT index 67 to 89
Replacing word [13624]= 67, SHIFT index 67 to 89
Replacing word [13626]= 67, SHIFT index 67 to 89
Replacing word [13629]= 67, SHIFT index 67 to 89
Replacing word [13634]= 67, SHIFT index 67 to 89
Replacing word [13640]= 67, SHIFT index 67 to 89
Replacing word [13651]= 67, SHIFT index 67 to 89
Replacing word [13652]= 67, SHIFT index 67 to 89
Replacing word [13653]= 67, SHIFT index 67 to 89
Replacing word [13657]= 67, SHIFT index 67 to 89
Replacing word [13658]= 67, SHIFT index 67 to 89
Replacing word [13659]= 67, SHIFT index 67 to 89
Replacing word [13661]= 67, SHIFT index 67 to 89
Replacing word [13662]= 67, SHIFT index 67 to 89
Replacing word [13665]= 67, SHIFT index 67 to 89
Replacing word [1366

In [9]:
from keras.utils import to_categorical
print(dm.root_dir_)
X, Y, vocab_mots, vocab_pdd, vocab_liaisons = dm.load_dummy_data_2()

print("|Y| ", len(Y))
print("|{Y}|", len(np.unique(Y)))
x = X[50025]
print("A sample : ")
print(x[0], x[1], x[2], x[3], x[4], " --> ", Y[50025])
print(vocab_mots[x[0]], vocab_mots[x[1]], vocab_pdd[x[2]], vocab_pdd[x[3]], x[4], " --> ", vocab_liaisons[Y[50025]])
print("vocab size: ", len(vocab_mots))

# treat data
X = np.array(X)
pdds = X[:,2]
print(X.shape)
print(pdds)
X_ = np.array(X)
cats_pdds_1 = to_categorical(X_[:, 2], num_classes=len(np.unique(vocab_pdd)))
cats_pdds_2 = to_categorical(X_[:, 3], num_classes=len(np.unique(vocab_pdd)))
min_dist = np.abs(np.min(X_[:, 4]))
cats_dist = to_categorical(X_[:, 4] + min_dist, num_classes=15)
print(cats_pdds_1)
print(cats_pdds_2.shape)
print(len(vocab_pdd))
print(np.unique(X_[:, 4]))
print(cats_dist.shape)

Y__ = to_categorical(Y)

X__ = np.column_stack((X_[:, 0], X_[:, 1], cats_pdds_1, cats_pdds_2, cats_dist))
print(X__.shape)
print(X__[50025])
#dm.safe_pickle_dump(os.path.join(dm.root_dir_, 'data', 'embeddings', 'facebookresearch', 'wiki.fr.pkl'), fr_embeddings)

#print(vocab_mots[vocab_mots])

C:\IAAA\tal-github
Loading data/f1_fr_project_ok_bool.pkl
|Y|  679243
|{Y}| 90
A sample : 
1085 7272 4 11 -1  -->  77
prend Matière VERB PROPN -1  -->  RIGHT_obj
vocab size:  42278
(679243, 6)
[18  0 18 ...  2  1  2]
[[0. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
(679243, 20)
20
[-7 -6 -5 -4 -3 -2 -1]
(679243, 15)
(679243, 57)
[1.085e+03 7.272e+03 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.00

In [10]:
print(np.unique(X_[:, 4]))

[-7 -6 -5 -4 -3 -2 -1]


In [4]:
print(X__[:, 2:X__.shape[1]])
print(Y__.shape)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(446113, 93)


In [5]:
word_vectors_fr = dm.load_embeddings('fr')
print("Loaded ", len(word_vectors_fr), " embeddings")

Loading C:\IAAA\tal-github\data\embeddings\facebookresearch\wiki.fr.vec.pkl
Embedding for fr loaded from C:\IAAA\tal-github\data\embeddings\facebookresearch\wiki.fr.vec.pkl
Read  1152449  words vectors
Loaded  1152449  embeddings


In [6]:

aligned_embeddings_fr, words_not_found_fr, words_matched_fr = dm.align_embeddings(vocab_mots, word_vectors_fr)

print("{found} words not found in pre-trained embeddings upon {total} from vocab"
      .format(found=len(words_not_found_fr), total=len(vocab_mots)))
print()
print("embeddings: ", aligned_embeddings_fr.shape)
print()
print("words not found:")
print([vocab_mots[w] for w in words_not_found_fr])


4546 words not found in pre-trained embeddings upon 42278 from vocab

embeddings:  (42278, 300)

words not found:
["qu'", "L'", 'vis-à-vis', "n'", "c'", 'petit-déjeuner', "d'", ';', "l'", 'abanbonné', "aujourd'hui", 'à-pic', ':', '1915', '93', '30e', '--', '155', '39', "s'", '105,3', '5633', '2010', '17e', '«', 'sous-espèce', '1527', 'Ymbrechts', "D'", 'A887', 'A87', 'Glenmoriston', '1986', 'Montsinéry-Tonnegrande', '...', '27', '2011', 'non-lieu', 'Paydrètes', 'celle-ci', 'ki-Moon', 'non-convexe', 'U59', "C'", "jusqu'", 'mi-août', '1984', '1983', '10 000', '48', '1962', 'États-Unis', 'Nouveau-Brunswick', 'Nouvelle-Angleterre', '1972', '846', '临济', 'Línjì', 'Yìxuán', '义', '玄', '869', '1991', '55', '1974', '1975', 'lui-même', '2', '1', 'Hadadah', '6', '10', '500', '1964', '1970', '1976', '1982', '1988', '1994', '2000', '2006', '69', '31', '50', '1936', '1987', '25', '322', '1934', 'mi-lourds', '1937', '1676', 'Crochais', '13', '81', '2004', '90', 'Gdeim', 'recludere', 'reclaudare', 'inc

In [7]:
print(vocab_mots.index('contre-feux'))
print('contre-feux' in word_vectors_fr.keys())
res = []
for w in word_vectors_fr.keys():
    if w.startswith('contre'):
        res.append(w)

print(sorted(res))

41239
False
['contre', 'contre,', 'contre/', 'contre/neutres', 'contre/pour', 'contre_plongeant_lethwei', 'contrebalance', 'contrebalancement', 'contrebalancent', 'contrebalancer', 'contrebalancera', 'contrebalancerait', 'contrebalancé', 'contrebalancée', 'contrebalancées', 'contrebalancés', 'contrebalançaient', 'contrebalançait', 'contrebalançant', 'contreband', 'contrebande', 'contrebandes', 'contrebandier', 'contrebandiers', 'contrebandiers\xa0»', 'contrebandière', 'contrebandières', 'contrebas', 'contrebasse', 'contrebasse,', 'contrebasses', 'contrebassine', 'contrebassiste', 'contrebassistes', 'contrebassite', 'contrebasson', 'contrebassons', 'contrebatterie', 'contrebattre', 'contrebia', 'contrebombarde', 'contrebutant', 'contrebutement', 'contrebutements', 'contrebutent', 'contrebuter', 'contrebuté', 'contrebutée', 'contrebutées', 'contrebutés', 'contrecarra', 'contrecarraient', 'contrecarrait', 'contrecarrant', 'contrecarre', 'contrecarrent', 'contrecarrer', 'contrecarrèrent', 

In [8]:

# free some memory as we don't need original embeddings anymore normally
del word_vectors_fr


In [12]:
import os
import pickle
import numpy as np
import pandas as pd

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Concatenate, Embedding, concatenate, Flatten, Dropout
from keras.engine.input_layer import Input
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split

class RNNManager:
    
    models_ = {}
    networks_ = {}
    path_ = '.'
    
    embeddings_ = None
    embeddings_for_words_ = None
    embeddings_for_lemmas_ = None
    
    dm_ = DataManager()
    
    def __init__(self, path='.'):
        if path != '.':
            os.makedirs(os.path.dirname(path), exist_ok=True)
        self.path_ = path
        self.dm_ = DataManager()
        
    def get_model(self, model_name):
        """
        Returns an existing TAL model, or None.
        
        Parameters
        ----------
        
        model_name: str
            Name of the model.
        """
        if model_name not in self.models_.keys():
            print("Model {m} not found".format(m=model_name))
            return None
        return self.models_[model_name]
        
    def create_model(self, model_name, lang, featureset, vocabs, vocabs_dev, vocabs_test, embeddings_file=None):
        """
        Creates a new TAL model.
        
        Parameters
        ----------
        
        model_name: str
            Name of this model.
            
        lang: str
            'fr', 'ja', 'nl', 'en'
            
        featureset: str
            'f1', 'f2' or 'f3'
            
        vocabs: dict
            Vocabs for learning task, with keys 'WORDS', 'POS', 'MORPHO' and/or 'LEMMA' (or 'LABELS' for targets).
        
        """
        model = {'name': model_name,
                'lang': lang,
                'featureset': featureset,
                'vocabs': vocabs,
                'vocabs_dev': vocabs_dev,
                'vocabs_test': vocabs_test,
                'embeddings_file': embeddings_file}
        if model_name not in self.models_.keys():
            self.models_[model_name] = model
            
        return model
            
    def remove_model(self, model_name):
        """
        """
        if model_name in self.models_.keys():
            return self.models_.pop(model_name, None)
    
    def save_model(self, model_name):
        """
        """
        if model_name in self.models_:
            self.dm_.safe_pickle_dump(os.path.join(self.path_, model_name + '-model.pkl'), self.models_[model_name])
        else:
            print("Model " + model_name + " not found")
            
    def load_model(self, model_fname):
        """
        """
        model_conf = self.dm_.pickle_load(os.path.join(self.path_, model_name, '-model.pkl'))
        if model:
            self.create_model(model_name=model['name'], lang=model['lang'], featureset=model['featureset'], 
                              vocabs=model['vocabs'])
            return 
            
        
    def create_network(self, network_name, model_name, nb_classes, embeddings=None, dropout=False):
        """
        
        
        Parameters
        ----------
        
        language: 'fr', 'ja', 'nl', 'en'
            For french, japanese, dutch, or english. 
        
        featureset: 'f1', 'f2', 'f3'
            
        
        embeddings: str
            If None, then pretrained embeddings are not used
            If 'words', then pretrained embeddings layers (and inputs) are added for 2 words
            If 'lemmas', then pretrained embeddings layers (and inputs) are added for 2 words and 2 lemmas
            if not None, then prepare_embeddings() must have been called prior to this to load pretrained embeddings.
        """
        net_model = None
        
        if model_name not in self.models_.keys():
            print("Model {m} not found".format(m=model_name))
            return None
        
        model = self.models_[model_name]
        featureset = model['featureset']
        lang = model['lang']
        vocabs = model['vocabs']
        
        net_model = Model()
                       
        #if embeddings != None:
            
        embedsw = self.embeddings_for_words_
        embedsl = self.embeddings_for_lemmas_
        
        if embedsw is not None:
            dim_embeddings = embedsw.shape[1]
        else:
            dim_embeddings = 300
            
        if embeddings is not None:
            
            if len(vocabs['WORDS']) != embedsw.shape[0]:
                print("Words vocab size {v} must equal embeddings length {e}".format(v=len(vocabs['WORDS']), 
                                                                               e=embedsw.shape[0]))
                return None
            
            # Pretrained Embedding layer for 2 words
            input_word1 = Input(shape=(1,), dtype='int32', name='word_input_1')
            embeddings_layer = Embedding(
                input_dim=len(vocabs['WORDS']), 
                output_dim=dim_embeddings, 
                weights=[embedsw], 
                input_length=1)(input_word1)
            #embeddings_1 = embeddings_layer(input_word1)
            embeddings_1 = Flatten()(embeddings_layer)
                
            # Embedding layer for second word
            input_word2 = Input(shape=(1,), dtype='int32', name='word_input_2')
            embeddings_2 = Embedding(input_dim=len(vocabs['WORDS']), 
                                 output_dim=dim_embeddings, 
                                 weights=[embedsw], 
                                 input_length=1)(input_word2)
            #embeddings_2 = embeddings_layer(input_word2) # sharing weights between both words embeddings
            embeddings_2 = Flatten()(embeddings_2)
            
        else:
            # Embedding layer for 2 words
            input_word1 = Input(shape=(1,), dtype='int32', name='word_input_1')
            embeddings_layer = Embedding(
                input_dim=len(vocabs['WORDS']), 
                output_dim=dim_embeddings, 
                input_length=1)(input_word1)
            embeddings_1 = Flatten()(embeddings_layer)
                
            # Embedding layer for second word
            input_word2 = Input(shape=(1,), dtype='int32', name='word_input_2')
            embeddings_2 = Embedding(input_dim=len(vocabs['WORDS']), 
                                 output_dim=dim_embeddings, 
                                 input_length=1)(input_word2)
            embeddings_2 = Flatten()(embeddings_2)
            
        if featureset is not 'f1' and embeddings == 'lemmas':
            # we must define also inputs and embeddings for lemmas
            if len(vocabs['LEMMA']) != embedsl.shape[0]:
                print("Lemma vocab size {v} must equal embeddings length {e}".format(v=len(vocabs['LEMMA']), 
                                                                               e=embedsl.shape[0]))
                return None
            
            # Pretrained Embedding layer for 2 lemmas
            input_lemma1 = Input(shape=(1,), dtype='int32', name='lemma_input_1')
            embeddingsl_layer = Embedding(
                input_dim=len(vocabs['LEMMA']), 
                output_dim=dim_embeddings, 
                weights=[embedsl], 
                input_length=1)(input_lemma1)
            embeddingsl_1 = Flatten()(embeddingsl_layer)
                
            # Embedding layer for second word
            input_lemma2 = Input(shape=(1,), dtype='int32', name='lemma_input_2')
            embeddingsl_2 = Embedding(
                input_dim=len(vocabs['LEMMA']), 
                output_dim=dim_embeddings, 
                weights=[embedsl], 
                input_length=1)(input_lemma2)
            embeddingsl_2 = Flatten()(embeddingsl_2)
            
        elif featureset is not 'f1':
            
            # Embedding layer for 2 lemmas
            input_lemma1 = Input(shape=(1,), dtype='int32', name='lemma_input_1')
            embeddingsl_layer = Embedding(
                input_dim=len(vocabs['LEMMA']), 
                output_dim=dim_embeddings, 
                input_length=1)(input_lemma1)
            embeddingsl_1 = Flatten()(embeddingsl_layer)
                
            # Embedding layer for second word
            input_lemma2 = Input(shape=(1,), dtype='int32', name='lemma_input_2')
            embeddingsl_2 = Embedding(
                input_dim=len(vocabs['LEMMA']), 
                output_dim=dim_embeddings, 
                input_length=1)(input_lemma2)
            embeddingsl_2 = Flatten()(embeddingsl_2)            
            
        
        # define input for additional features
        # note: dist is restricted to [-7 ... 7] so 15 values
        if featureset == 'f1':
            """ S.0.POS
                B.0.POS
                DIST"""
            dim_features = len(vocabs['POS']) * 2 + 8
        elif featureset == 'f2':
            """ S.0.POS
                S.0.LEMMA
                S.0.MORPHO
                S.-1.POS
                B.0.POS
                B.0.LEMMA
                B.0.MORPHO
                B.-1.POS
                B.1.POS
                DIST"""
            dim_features = len(vocabs['POS']) * 4 + len(vocabs['MORPHO']) * 2 + 8 # + len(vocabs['LEMMA']) * 2
        else:
            """ S.0.POS
                S.0.LEMMA
                S.0.MORPHO
                S.1.POS
                B.0.POS
                B.0.LEMMA
                B.0.MORPHO
                B.-1.POS
                B.1.POS
                DIST"""
            # same size as 'f2'
            dim_features = len(vocabs['POS']) * 4 + len(vocabs['MORPHO']) * 2 + 8 # + len(vocabs['LEMMA']) * 2
        print("  expecting {n} features dim".format(n=dim_features))
        
        # define input for features
        features_input = Input(shape=(dim_features,))
        
        # define concatenation depending on embeddings and features
        
        if embeddings is not None:
            if featureset is not 'f1' and embeddings == 'lemmas':
                # concatenate features and embeddings
                l = concatenate([embeddings_1, embeddings_2, embeddings_l1, embeddings_l2, features_input])
            else:
                # concatenate features and embeddings
                l = concatenate([embeddings_1, embeddings_2, features_input])
        else:
            l = features_input
        
        # adding dense layers
        
        l = Dense(128)(l)
        l = Activation('relu')(l)
        if dropout:
            l = Dropout(0.15)(l)
        l = Dense(128)(l)
        l = Activation('relu')(l)
        if dropout:
            l = Dropout(0.15)(l)
        
        l = Dense(nb_classes)(l)
        out = Activation('softmax')(l)
        
        if embeddings is not None:
            net_model = Model([input_word1, input_word2, features_input], out)
        else:
            net_model = Model(features_input, out)
        
        # not sure where to compile the model ...
        net_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
            
        if net_model:
            self.networks_[network_name] = net_model
            # save initial state of this network (this also saves embedding layers, so we remove them)
            self.save_network(network_name)
            #del self.embeddings_for_words_
            #del self.embeddings_for_lemmas_
        
        return net_model

    def save_network(self, network_name):
        if name in self.networks_:
            self.networks_[network_name].save(os.path.join(self.path_, '/', network_name, '-net.h5'))
        else:
            print("Net " + network_name + " not found")
      
    def preprocess_embeddings(self, model_name):
        """
        Preprocess embeddings for training a network for this model.
        Notes:
        - should be run once BEFORE creating a network, if you want to use pre-trained embeddings to train this network, 
          if not embeddings will be completely learned during training
        - loaded embeddings should be deleted before loading/running a new network to avoid exhausting memory.
        - if called again for another model embeddings will be REPLACED
        - for already created network embedded remain and will be saved along with the network (no need to call this if
          you load a saved network)
            - at network creation, once net model is saved to disk with keras model.save(), loaded embeddings are deleted
              to free memory
            - to manually remove loaded embeddings, delete attributes embeddings_, embeddings_for_words_ and embeddings_for_lemmas_
        
        Parameters
        ----------
        
        model_name: str
            Name of the TAL model.
            
        Returns: nothing
        
        """
        
        if model_name not in self.models_.keys():
            print("Model {m} not found".format(m=model_name))
            return None
        model = self.models_[model_name]
        featureset = model['featureset']
        lang = model['lang']
        vocabs = model['vocabs']
        
        if self.embeddings_for_words_ is None or (featureset is not 'f1' and self.embeddings_for_lemmas_ is None):
        
            if self.embeddings_ is None:
                self.embeddings_ = dm.load_embeddings(lang)
            
            if self.embeddings_for_words_ is None:
                self.embeddings_for_words_, words_not_found, words_matched = dm.align_embeddings(
                    vocabs['WORDS'], self.embeddings_)
            
            if featureset is not 'f1' and self.embeddings_for_lemmas_ is None:
                self.embeddings_for_lemmas_, words_not_found, words_matched = dm.align_embeddings(
                    vocabs['LEMMA'], self.embeddings_)
       
        # free some memoy
        del self.embeddings_
        
        return
    
            
    def preprocess_data(self, model_name, X_train, y_train, X_dev, y_dev, X_test, y_test):
        """
        Preprocess data for training a network for this model.
        - merges vocabs (WORDS) of X_dev into X_train
        - merges vocabs (LEMMA) of X_dev into X_train, if any
        - merges vocabs (MORPHO) of X_dev into X_train, if any
        - merges vocabs (labels) of Y_dev and Y_test into Y_train
        - converts to categorical (one-hot), features of X_* (POS, MORPHO, DIST)
        
        Note: arrays and vocabs are updated in place, so keep a copy of originals if required.
        
        Parameters
        ----------
        
        model_name: str
            TAL model to use for treating data (defines features and vocabs).
            
        X_train, y_train, X_dev, y_dev, X_test, y_test: arrays
            Training, validation and test data to process.
            
        Returns
        -------
        
        X_train, y_train, X_dev, y_dev, X_test, y_test
            
        """
        
        if model_name not in self.models_.keys():
            print("Model {m} not found".format(m=model_name))
            return None
        model = self.models_[model_name]
        featureset = model['featureset']
        lang = model['lang']
        vocabs = model['vocabs']
        vocabs_dev = model['vocabs_dev']
        vocabs_test = model['vocabs_test']
        
        # Treat vocabularies
        
        print("preprocess_data: merging WORDS vocabs from dev into train ...")
        self.dm_.merge_vocabs(vocab1=vocabs['WORDS'], vocab2=vocabs_dev['WORDS'], data=X_dev, columns=(0, 1))
        # ... vocabs_dev['WORDS'] is now useless
        
        if 'POS' in vocabs:
            nb_classes_pos = len(np.unique(vocabs['POS']))
        if 'MORPHO' in vocabs:
            print("preprocess_data: merging and aligning MORPHO vocabs from dev into train ...")
            self.dm_.merge_vocabs(vocab1=vocabs['MORPHO'], vocab2=vocabs_dev['MORPHO'], data=X_dev, columns=(4, 8))
            # ... vocabs_dev['MORPHO'] is now useless
            nb_classes_morpho = len(np.unique(vocabs['MORPHO']))
        if 'LEMMA' in vocabs:
            print("preprocess_data: merging and aligning LEMMA vocabs from dev into train ...")
            self.dm_.merge_vocabs(vocab1=vocabs['LEMMA'], vocab2=vocabs_dev['LEMMA'], data=X_dev, columns=(3, 7))
            # ... vocabs_dev['LEMMA'] is now useless
        
        print(vocabs.keys())
        print(vocabs_dev.keys())
        print("preprocess_data: merging and aligning LABEL vocabs from dev into train ...")
        self.dm_.merge_vocabs(vocab1=vocabs['LABELS'], vocab2=vocabs_dev['LABELS'], data=y_dev, columns=(0,))
        print("preprocess_data: merging and aligning LABEL vocabs from test into train ...")
        self.dm_.merge_vocabs(vocab1=vocabs['LABELS'], vocab2=vocabs_test['LABELS'], data=y_test, columns=(0,))
        nb_classes = len(vocabs['LABELS'])
            
        # convert some features to one-hot encoding
        
        nb_classes_dist = 8
        
        
        print("preprocess_data: converting {f} features to one-hot encoding...".format(f=featureset))
        
        if featureset == 'f1':
            cats_pos1 = to_categorical(X_train[:, 2], num_classes=nb_classes_pos)
            cats_pos2 = to_categorical(X_train[:, 3], num_classes=nb_classes_pos)
            # dist is positive for arc eager
            cats_dist = to_categorical(np.abs(X_train[:, 4]), num_classes=nb_classes_dist)
            X_train = np.column_stack((X_train[:, 0], X_train[:, 1], cats_pos1, cats_pos2, cats_dist))
            
            cats_pos1 = to_categorical(X_dev[:, 2], num_classes=nb_classes_pos)
            cats_pos2 = to_categorical(X_dev[:, 3], num_classes=nb_classes_pos)
            # dist is positive for arc eager
            cats_dist = to_categorical(np.abs(X_dev[:, 4]), num_classes=nb_classes_dist)
            X_dev = np.column_stack((X_dev[:, 0], X_dev[:, 1], cats_pos1, cats_pos2, cats_dist))
            
            cats_pos1 = to_categorical(X_test[:, 2], num_classes=nb_classes_pos)
            cats_pos2 = to_categorical(X_test[:, 3], num_classes=nb_classes_pos)
            # dist is positive for arc eager
            cats_dist = to_categorical(np.abs(X_test[:, 4]), num_classes=nb_classes_dist)
            X_test = np.column_stack((X_test[:, 0], X_test[:, 1], cats_pos1, cats_pos2, cats_dist))
            
        else: # 'f2' and 'f3' featuresets have same structure
            """ 
            S.0.POS
            S.0.LEMMA
            S.0.MORPHO
            S.-1.POS (f2, or S.1.POS for f3)
            B.0.POS
            B.0.LEMMA
            B.0.MORPHO
            B.-1.POS
            B.1.POS
            DIST"""                
            cats_pos1    = to_categorical(X_train[:, 2], num_classes=nb_classes_pos)
            cats_morpho1 = to_categorical(X_train[:, 4], num_classes=nb_classes_morpho)
            cats_pos2    = to_categorical(X_train[:, 5], num_classes=nb_classes_pos)
            cats_pos3    = to_categorical(X_train[:, 6], num_classes=nb_classes_pos)
            cats_morpho2 = to_categorical(X_train[:, 8], num_classes=nb_classes_morpho)
            cats_pos4    = to_categorical(X_train[:, 9], num_classes=nb_classes_pos)
            cats_pos5    = to_categorical(X_train[:, 10], num_classes=nb_classes_pos)
            cats_dist    = to_categorical(np.abs(X_train[:, 11]), num_classes=nb_classes_dist)
            X_train = np.column_stack((X_train[:, 0], X_train[:, 1], cats_pos1, X_train[:, 3],
                                      cats_morpho1, cats_pos2, cats_pos3,
                               X_train[:, 7], cat_trains_morpho2, cats_pos4, cats_pos5, cats_dist))
            
            cats_pos1    = to_categorical(X_dev[:, 2], num_classes=nb_classes_pos)
            cats_morpho1 = to_categorical(X_dev[:, 4], num_classes=nb_classes_morpho)
            cats_pos2    = to_categorical(X_dev[:, 5], num_classes=nb_classes_pos)
            cats_pos3    = to_categorical(X_dev[:, 6], num_classes=nb_classes_pos)
            cats_morpho2 = to_categorical(X_dev[:, 8], num_classes=nb_classes_morpho)
            cats_pos4    = to_categorical(X_dev[:, 9], num_classes=nb_classes_pos)
            cats_pos5    = to_categorical(X_dev[:, 10], num_classes=nb_classes_pos)
            cats_dist    = to_categorical(np.abs(X_dev[:, 11]), num_classes=nb_classes_dist)
            X_dev = np.column_stack((X_dev[:, 0], X_dev[:, 1], cats_pos1, X_dev[:, 3],
                                      cats_morpho1, cats_pos2, cats_pos3,
                               X_dev[:, 7], cat_trains_morpho2, cats_pos4, cats_pos5, cats_dist))
            

            cats_pos1    = to_categorical(X_test[:, 2], num_classes=nb_classes_pos)
            cats_morpho1 = to_categorical(X_test[:, 4], num_classes=nb_classes_morpho)
            cats_pos2    = to_categorical(X_test[:, 5], num_classes=nb_classes_pos)
            cats_pos3    = to_categorical(X_test[:, 6], num_classes=nb_classes_pos)
            cats_morpho2 = to_categorical(X_test[:, 8], num_classes=nb_classes_morpho)
            cats_pos4    = to_categorical(X_test[:, 9], num_classes=nb_classes_pos)
            cats_pos5    = to_categorical(X_test[:, 10], num_classes=nb_classes_pos)
            cats_dist    = to_categorical(np.abs(X_test[:, 11]), num_classes=nb_classes_dist)
            X_test = np.column_stack((X_test[:, 0], X_test[:, 1], cats_pos1, X_test[:, 3],
                                      cats_morpho1, cats_pos2, cats_pos3,
                               X_test[:, 7], cats_morpho2, cats_pos4, cats_pos5, cats_dist))
                        
        print("preprocess_data: converting LABELs to one-hot encoding ...")
        y_train = to_categorical(y_train, num_classes=nb_classes)
        y_dev = to_categorical(y_dev, num_classes=nb_classes)
        y_test = to_categorical(y_test, num_classes=nb_classes)
                
        return X_train, y_train, X_dev, y_dev, X_test, y_test
    

Error in callback <bound method Beeper.post_execute of <__main__.Beeper object at 0x00000256F97B67B8>> (for post_execute):


TypeError: 'module' object is not callable

In [11]:
import time

t = time.time()

print()

X_train, y_train, vocabs_train = dm.load_data()
X_dev, y_dev, vocabs_dev = dm.load_data(phase='dev')
X_test, y_test, vocabs_test = dm.load_data(phase='test')

print("loaded data in ", time.time() - t)

nb_classes = len(vocabs_train['LABELS'])

lang = 'fr'
featureset = 'f1'
model_name = "{featureset}_{lang}-model".format(featureset=featureset, lang=lang)

# testing the TALRNNManager class
rnn = RNNManager()

# create a TAL model for this combination
rnn.remove_model(model_name)
rnn.create_model(model_name, lang, featureset, vocabs_train, vocabs_dev, vocabs_test)
rnn.save_model(model_name)

t = time.time()

print()
# preprocess X/Y data
X_train, y_train, X_dev, y_dev, X_test, y_test = rnn.preprocess_data(model_name, X_train, y_train, X_dev, y_dev, X_test, y_test)

print("Preprocessed data in ", time.time() - t)

t = time.time()
print()
# load pretrained embeddings
rnn.preprocess_embeddings(model_name)

print("Preprocessed embeddings in ", time.time() - t)

t = time.time()
print()
# create a classifier for this TAL model
network_name = "{model}-net".format(model=model_name)
net = rnn.create_network(network_name, model_name, nb_classes=nb_classes, embeddings='words', dropout=True)

print("Created net in ", time.time() - t)

print(rnn.networks_)


Loading data\f1_fr-train
load_data: loaded X =  (679243, 6) , Y =  (679243,) , vocabs =  WORDS (42278), POS (20), LABELS (90), 
Loading data\f1_fr-dev
load_data: loaded X =  (68105, 6) , Y =  (68105,) , vocabs =  WORDS (9275), POS (20), LABELS (74), 
Loading data\f1_fr-test
load_data: loaded X =  (19119, 6) , Y =  (19119,) , vocabs =  WORDS (3285), POS (20), LABELS (68), 
loaded data in  1.5861165523529053
Saving .\f1_fr-model-model.pkl

preprocess_data: merging WORDS vocabs from dev into train ...
dict_keys(['WORDS', 'POS', 'LABELS'])
dict_keys(['WORDS', 'POS', 'LABELS'])
preprocess_data: merging and aligning LABEL vocabs from dev into train ...
preprocess_data: merging and aligning LABEL vocabs from test into train ...
preprocess_data: converting f1 features to one-hot encoding...
preprocess_data: converting LABELs to one-hot encoding ...
Preprocessed data in  580.4876592159271

Loading C:\IAAA\tal-github\data\embeddings\facebookresearch\wiki.fr.vec.pkl
Embedding for fr loaded from 

NameError: name 'save_network' is not defined

Error in callback <bound method Beeper.post_execute of <__main__.Beeper object at 0x00000256F97B67B8>> (for post_execute):


TypeError: 'module' object is not callable

In [8]:
net = rnn.create_network(network_name, model_name, nb_classes=nb_classes, embeddings='words', dropout=True)

TypeError: 'Tensor' object is not callable

Error in callback <bound method Beeper.post_execute of <__main__.Beeper object at 0x00000256F97B67B8>> (for post_execute):


TypeError: 'module' object is not callable

In [12]:
history = net.fit([X_train[:, 0], X_train[:, 1], X_train[:, 2:X_train.shape[1]]], 
                  y_train,
                  validation_data=(
                      [X_dev[:, 0], X_dev[:, 1], X_dev[:, 2:X_dev.shape[1]]], 
                      y_dev)
                  epochs=10)

Epoch 1/10
446113/446113 [==============================] - 661s 1ms/step - loss: 0.6085 - acc: 0.7874
Epoch 2/10
446113/446113 [==============================] - 658s 1ms/step - loss: 0.5180 - acc: 0.8190
Epoch 3/10
446113/446113 [==============================] - 659s 1ms/step - loss: 0.4499 - acc: 0.8433
Epoch 4/10
446113/446113 [==============================] - 659s 1ms/step - loss: 0.4023 - acc: 0.8598
Epoch 5/10
446113/446113 [==============================] - 657s 1ms/step - loss: 0.3674 - acc: 0.8709
Epoch 6/10
446113/446113 [==============================] - 658s 1ms/step - loss: 0.3413 - acc: 0.8792
Epoch 7/10
446113/446113 [==============================] - 660s 1ms/step - loss: 0.3205 - acc: 0.8865
Epoch 8/10
446113/446113 [==============================] - 662s 1ms/step - loss: 0.3065 - acc: 0.8905 0s - loss: 0.3064 - acc:
Epoch 9/10
446113/446113 [==============================] - 659s 1ms/step - loss: 0.2932 - acc: 0.8960
Epoch 10/10
446113/446113 [=====================

In [ ]:
rnn.save_network(network_name)